In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from func import get_t_shirt, add_to_cart, place_order
from IPython.display import Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4.1"

client = OpenAI(
    api_key=api_key
)

In [4]:
tools = [
    {
        "type": "function",
        "name": "get_t_shirts",
        "description": "Use this tool to get the list of available t-shirts",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the t-shirt to retrieve from the database"
                },
                "size": {
                    "type": "string",
                    "description": "The size of the t-shirt to retrieve from the database"
                },
                "color": {
                    "type": "string",
                    "description": "The color of the t-shirt to retrieve from the database"
                }
            },
            "required": ["name", "size", "color"],
            "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "add_to_cart",
        "description": "Use this tool to add the customer's user to the cart. If the customer wants more than one, provide the aggregated price into the parameters.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the t-shirt to add to the cart"
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the t-shirt to add to the cart"
                },
                "size": {
                    "type": "string",
                    "description": "The size of the t-shirt to add to the cart"
                },
                "color": {
                    "type": "string",
                    "description": "The color of the t-shirt to add to the cart"
                },
                "price": {
                    "type": "string",
                    "description": "The price of the t-shirt to add to the cart"
                }
            },
            "required": ["name", "size", "color", "quantity", "price"],
            "additionalProperties": False
        },
        "strict": True,
    },
    {
        "type": "function",
        "name": "place_order",
        "description": "Use this tool to place the order for the customer's items in his or her cart.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the t-shirt to add to their order"
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the t-shirt to add to their order"
                },
                "size": {
                    "type": "string",
                    "description": "The size of the t-shirt to add to their order"
                },
                "color": {
                    "type": "string",
                    "description": "The color of the t-shirt to add to their order"
                },
                "price": {
                    "type": "string",
                    "description": "The price of the t-shirt to add to their order"
                }
            },
            "required": ["name", "size", "color", "quantity", "price"],
            "additionalProperties": False
        },
        "strict": True,
    }
]

In [5]:
function_map = {
    "get_t_shirts": get_t_shirt,
    "add_to_cart": add_to_cart,
    "place_order": place_order
}

In [6]:
def get_completions(input_messages, model=model, tools=tools):
    llm_response = client.responses.create(
        model=model,
        input=input_messages,
        tools=tools,
    )

    output = llm_response.output[0]
    return output

In [7]:
input_messages = [
    {
        "role": "system",
        "content": "You are an action-oriented assistant that helps users with t-shirt orders. Your tasks include checking inventory, adding items to the cart, and placing orders. Always provide clear and complete answers. When a user wants to order a shirt, confirm its availability. If it’s in stock, offer to add it to their cart. After that, ask if they'd like to place the order."
    }
]

In [8]:
while True:
    user_input = input("👤 You: ")
    
    if user_input.lower() in {"exit", "quit"}:
        print("👋 Exiting conversation.")
        break

    print(f"👤 (You): {user_input}")
    input_messages.append({
        "role": "user", 
        "content": user_input
        })
    
    response = get_completions(input_messages)

    while response.type== "function_call":
        func_name = response.name
        input_messages.append(response)
        
        if func_name in function_map:
            args = json.loads(response.arguments)
            result = function_map[func_name](**args)

            input_messages.append({
                "type": "function_call_output",
                "call_id": response.call_id,
                "output": str(result),
            })

            response = get_completions(input_messages)
        else:
            print(f"Unknown function {func_name}")
    
    if response.type == "message":
        print(f"🤖: {response.content[0].text}")
        input_messages.append({
            "role": "assistant",
            "content": response.content[0].text
            })        

👤 (You): Hi!
🤖: Hello! How can I assist you with t-shirt orders today?
👤 (You): I'd like to order the 'My AI is Smarter Than Your Honor Student' t-shirt in size M and color White please.
🤖: The "My AI is Smarter Than Your Honor Student" t-shirt in size M and color White is available for $19.99. Would you like me to add it to your cart? If you need more than one, please let me know the quantity!
👤 (You): Please add 2 into my cart.
🤖: I’ve added 2 "My AI is Smarter Than Your Honor Student" t-shirts (size M, color White) to your cart for a total of $39.98.

Would you like to place the order now or continue shopping?
👤 (You): Place my order please.
🤖: Your order for 2 "My AI is Smarter Than Your Honor Student" t-shirts (size M, color White) has been placed for a total of $39.98.

Thank you for your order! Would you like a confirmation email or need help with anything else?
👤 (You): Thanks!
🤖: You're welcome! If you need anything else or want to place another order in the future, just let m

In [10]:
# see the full converation here

input_messages_clean = [
    msg.model_dump() if hasattr(msg, "model_dump") else msg
    for msg in input_messages
]

print(json.dumps(input_messages_clean, indent=2))

[
  {
    "role": "system",
    "content": "You are an action-oriented assistant that helps users with t-shirt orders. Your tasks include checking inventory, adding items to the cart, and placing orders. Always provide clear and complete answers. When a user wants to order a shirt, confirm its availability. If it\u2019s in stock, offer to add it to their cart. After that, ask if they'd like to place the order."
  },
  {
    "role": "user",
    "content": "Hi!"
  },
  {
    "role": "assistant",
    "content": "Hello! How can I assist you with t-shirt orders today?"
  },
  {
    "role": "user",
    "content": "I'd like to order the 'My AI is Smarter Than Your Honor Student' t-shirt in size M and color White please."
  },
  {
    "arguments": "{\"name\":\"My AI is Smarter Than Your Honor Student\",\"size\":\"M\",\"color\":\"White\"}",
    "call_id": "call_6iIyP00zrb4eJ7Rec6uWhcnt",
    "name": "get_t_shirts",
    "type": "function_call",
    "id": "fc_6872523340e0819f9b04f5ca3f6f90770da7e